# Direct Preference Optimization (DPO) Training with SageMaker

This notebook demonstrates how to use the **DPOTrainer** to fine-tune large language models using Direct Preference Optimization (DPO). DPO is a technique that trains models to align with human preferences by learning from preference data without requiring a separate reward model.

## What is DPO?

Direct Preference Optimization (DPO) is a method for training language models to follow human preferences. Unlike traditional RLHF (Reinforcement Learning from Human Feedback), DPO directly optimizes the model using preference pairs without needing a reward model.

**Key Benefits:**
- Simpler than RLHF - no reward model required
- More stable training process
- Direct optimization on preference data
- Works with LoRA for efficient fine-tuning

## Workflow Overview

1. **Prepare Preference Dataset**: Upload preference data in JSONL format
2. **Register Dataset**: Create a SageMaker AI Registry dataset
3. **Configure DPO Trainer**: Set up model, training parameters, and resources
4. **Execute Training**: Run the DPO fine-tuning job
5. **Track Results**: Monitor training with MLflow integration

## Step 1: Prepare and Register Preference Dataset

DPO requires preference data in a specific format where each example contains:
- **prompt**: The input text
- **chosen**: The preferred response
- **rejected**: The less preferred response

The dataset should be in JSONL format with each line containing one preference example.

In [5]:
from sagemaker.ai_registry.dataset import DataSet
from sagemaker.ai_registry.dataset_utils import CustomizationTechnique

# Upload dataset to S3
import boto3
s3 = boto3.client('s3')
s3.upload_file(
    './dpo-preference_dataset_train_256.jsonl',
    'nova-mlflow-us-west-2',
    'dataset/preference_dataset_train_256.jsonl'
)

# Register dataset in SageMaker AI Registry
# This creates a versioned dataset that can be referenced by ARN
dataset = DataSet.create(
    name="demo-nargokul-6", 
    data_location="s3://nova-mlflow-us-west-2/dataset/preference_dataset_train_256.jsonl", 
    customization_technique=CustomizationTechnique.DPO, 
    wait=True
)

print(f"Dataset ARN: {dataset.arn}")

Output()

Final Resource Status: Available

Dataset ARN: arn:aws:sagemaker:us-west-2:052150106756:hub-content/F3LMYANDKWPZCROJVCKMJ7TOML6QMZBZRRQOVTUL45VUK7PJ4SXA/DataSet/demo-nargokul-6/0.0.4


## Step 2: Configure and Execute DPO Training

The **DPOTrainer** provides a high-level interface for DPO fine-tuning with the following key features:

### Key Parameters:
- **model**: Base model to fine-tune (from SageMaker Hub)
- **training_type**: Fine-tuning method (LoRA recommended for efficiency)
- **training_dataset**: ARN of the registered preference dataset
- **model_package_group_name**: Where to store the fine-tuned model
- **mlflow_resource_arn**: MLflow tracking server for experiment logging

### Training Features:
- **Serverless Training**: Automatically managed compute resources
- **LoRA Integration**: Parameter-efficient fine-tuning
- **MLflow Tracking**: Automatic experiment and metrics logging
- **Model Versioning**: Automatic model package creation

In [ ]:
import random
! ada credentials update --provider=isengard --account=052150106756 --role=Admin --profile=default --once
! aws configure set region  us-west-2

from sagemaker.train.dpo_trainer import DPOTrainer
from sagemaker.train.common import TrainingType



In [2]:
# Create DPOTrainer instance with comprehensive configuration
trainer = DPOTrainer(
    # Base model from SageMaker Hub
    model="meta-textgeneration-llama-3-2-1b-instruct",
    
    # Use LoRA for efficient fine-tuning
    training_type=TrainingType.LORA,
    
    # Model versioning and storage
    model_package_group_name="arn:aws:sagemaker:us-west-2:052150106756:model-package-group/test-finetuned-models-gamma",
    
    # MLflow experiment tracking
    mlflow_resource_arn="arn:aws:sagemaker:us-west-2:052150106756:mlflow-tracking-server/ashwpat-test",
    
    # Training data (from Step 1)
    training_dataset="arn:aws:sagemaker:us-west-2:052150106756:hub-content/F3LMYANDKWPZCROJVCKMJ7TOML6QMZBZRRQOVTUL45VUK7PJ4SXA/DataSet/demo-nargokul-6/0.0.4",
    
    # Output configuration
    s3_output_path="s3://nova-mlflow-us-west-2/output",
    
    # IAM role for training job
    role="arn:aws:iam::052150106756:role/Admin",
    
    # Unique job name
    base_job_name=f"dpo-llama-{random.randint(1, 1000)}",
)

# Customize training hyperparameters
# DPO-specific parameters are automatically loaded from the model's recipe
trainer.hyperparameters.max_epochs = 1  # Quick training for demo

print("Starting DPO training job...")
print(f"Job name: {trainer.base_job_name}")
print(f"Base model: {trainer._model_name}")

# Execute training with monitoring
training_job = trainer.train(wait=True)

print(f"Training completed! Job ARN: {training_job.training_job_arn}")

[11/24/25 17:08:50] INFO     SageMaker session not provided. Using default Session.                  ]8;id=142678;file:///Users/rsareddy/workplace/virtual_envs/sagemaker-v3/lib/python3.12/site-packages/sagemaker/train/defaults.py\defaults.py]8;;\:]8;id=446735;file:///Users/rsareddy/workplace/virtual_envs/sagemaker-v3/lib/python3.12/site-packages/sagemaker/train/defaults.py#61\61]8;;\

Starting DPO training job...
Job name: dpo-llama-721
Base model: meta-textgeneration-llama-3-2-1b-instruct


[11/24/25 17:08:51] INFO     SageMaker session not provided. Using default Session.                  ]8;id=911996;file:///Users/rsareddy/workplace/virtual_envs/sagemaker-v3/lib/python3.12/site-packages/sagemaker/train/defaults.py\defaults.py]8;;\:]8;id=58495;file:///Users/rsareddy/workplace/virtual_envs/sagemaker-v3/lib/python3.12/site-packages/sagemaker/train/defaults.py#61\61]8;;\

                    INFO     Training Job Name: dpo-llama-721-20251124170851                     ]8;id=517485;file:///Users/rsareddy/workplace/virtual_envs/sagemaker-v3/lib/python3.12/site-packages/sagemaker/train/dpo_trainer.py\dpo_trainer.py]8;;\:]8;id=652836;file:///Users/rsareddy/workplace/virtual_envs/sagemaker-v3/lib/python3.12/site-packages/sagemaker/train/dpo_trainer.py#115\115]8;;\

Training Job Name: dpo-llama-721-20251124170851


                    INFO     MLflow resource ARN:                                             ]8;id=293371;file:///Users/rsareddy/workplace/virtual_envs/sagemaker-v3/lib/python3.12/site-packages/sagemaker/train/common_utils/finetune_utils.py\finetune_utils.py]8;;\:]8;id=444970;file:///Users/rsareddy/workplace/virtual_envs/sagemaker-v3/lib/python3.12/site-packages/sagemaker/train/common_utils/finetune_utils.py#435\435]8;;\
                             arn:aws:sagemaker:us-west-2:052150106756:mlflow-tracking-server/                      
                             ashwpat-test                                                                          

                    INFO     Creating training_job resource.                                     ]8;id=617267;file:///Users/rsareddy/workplace/virtual_envs/sagemaker-v3/lib/python3.12/site-packages/sagemaker/core/resources.py\resources.py]8;;\:]8;id=485192;file:///Users/rsareddy/workplace/virtual_envs/sagemaker-v3/lib/python3.12/site-packages/sagemaker/core/resources.py#35539\35539]8;;\

Output()

[11/24/25 17:17:28] INFO     Final Resource Status: Completed                                    ]8;id=678286;file:///Users/rsareddy/workplace/virtual_envs/sagemaker-v3/lib/python3.12/site-packages/sagemaker/core/resources.py\resources.py]8;;\:]8;id=690969;file:///Users/rsareddy/workplace/virtual_envs/sagemaker-v3/lib/python3.12/site-packages/sagemaker/core/resources.py#35872\35872]8;;\

In [4]:
print(training_job)

training_job_name='dpo-llama-721-20251124170851' training_job_arn='arn:aws:sagemaker:us-west-2:052150106756:training-job/dpo-llama-721-20251124170851' processing_job_arn=<sagemaker.core.utils.utils.Unassigned object at 0x17deaa300> tuning_job_arn=<sagemaker.core.utils.utils.Unassigned object at 0x17deaa300> labeling_job_arn=<sagemaker.core.utils.utils.Unassigned object at 0x17deaa300> auto_ml_job_arn=<sagemaker.core.utils.utils.Unassigned object at 0x17deaa300> model_artifacts=ModelArtifacts(s3_model_artifacts='s3://nova-mlflow-us-west-2/output/dpo-llama-721-20251124170851/output/model') training_job_output=TrainingJobOutput(s3_training_job_output='s3://nova-mlflow-us-west-2/output/dpo-llama-721-20251124170851/output/output') training_job_status='Completed' secondary_status='Completed' failure_reason=<sagemaker.core.utils.utils.Unassigned object at 0x17deaa300> hyper_parameters={'adam_beta': '0.01', 'global_batch_size': '16', 'learning_rate': '0.0001', 'lora_alpha': '32', 'lr_warmup_ra

## Next Steps

After training completes, you can:

1. **Deploy the Model**: Use `ModelBuilder` to deploy the fine-tuned model
2. **Evaluate Performance**: Compare responses from base vs fine-tuned model
3. **Monitor Metrics**: Review training metrics in MLflow
4. **Iterate**: Adjust hyperparameters and retrain if needed

### Example Deployment:
```python
from sagemaker.serve import ModelBuilder

# Deploy the fine-tuned model
model_builder = ModelBuilder(model=training_job)
model_builder.build(role_arn="arn:aws:iam::account:role/SageMakerRole")
endpoint = model_builder.deploy(endpoint_name="dpo-finetuned-llama")
```

The fine-tuned model will now generate responses that better align with the preferences in your training data.